# Short demo

In [1]:
import sys
import importlib
from zkpytoolkit import ZKP

We always start by importing the compiler and initializing it (can only be done once as it generates a config stored in a OnceCell in Rust!)

In [2]:
zkp = ZKP("bls12_381", 0, "groth16")
# zkp = ZKP()

In [3]:
# Only import the field after instantiating the ZKP
from zkpytoolkit.types import Private, Public, Array, field

Let's start with simple matrix multiplication of two private 2x2 matrices over the specified finite field.

In [4]:
def mm(
    A: Private[Array[Array[field, 2], 2]],
    B: Private[Array[Array[field, 2], 2]],
) -> Array[Array[field, 2], 2]:
    AB: Array[Array[field, 2], 2] = [[field(0) for _ in range(2)] for _ in range(2)]
    for i in range(2):
        for j in range(2):
            for k in range(2):
                AB[i][j] = AB[i][j] + A[i][k] * B[k][j]
    return AB


We must first compile the function into an R1CS instance

In [5]:
constraints = zkp.compile(mm)

If the compilation is successful, we also retrieve the number of constraints from the R1CS

In [6]:
print(constraints)

8


We can now instantiate the prover and verifier statements, and see what a successful zero-knowledge proof looks like. First let us obtain the result of running the function with some private inputs

In [7]:
A = [[field(23), field(52)], [field(99), field(123)]]
B = [[field(12), field(1)], [field(100), field(42)]]
C = mm(A, B)
print(C)

[[5476, 2207], [13488, 5265]]


Next, we generate the CRS and the proof and then verify the proof

In [8]:
crs = zkp.generate_crs(mm)
zkp.store_crs(mm, crs) # optional if run locally
proof = zkp.prove(mm, A, B)
zkp.store_proof(mm, proof) # optional if run locally
zkp.verify(mm, A, B, return_value=C)

True

Finally, we clean up the prover and verifier keys and statements.

In [9]:
zkp.cleanup()

Now let us try a more sophisticated example, involving multiple functions and include statements. Let us verifiably compute a function that sums up an integer 3 times by using a sha256-based commitment scheme 

In [10]:
from zkpytoolkit.stdlib.commitment.sha256.commit import commit

def _3_plus_int(x: Private[int]) -> int:
    return x + x + x

# We are going to verifiably compute _3_plus_int(x) and
# compose that with a proof for the pre-image of sha256(x)
def verifiable_plus(x: Private[int], rand: Private[Array[int, 16]], x_comm: Public[Array[int, 8]]) -> int:
    x_packed: Array[int, 16] = [x, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    assert(x_comm == commit(x_packed, rand))
    return _3_plus_int(x)


Here we have that the commitment for x is public, which means that the verifier must have access to it. In practice, this means that the prover sends the verifier the commitment to x before even computing the target function. This allows the verifier to ensure that the prover is required to compute the function with the claimed value for x.

Let's suppose that the prover uses the correct commitment to prove that the 3 plus function was computed correctly

In [11]:
rand = [0xfeeae7f2,
        0x0114f06d,
        0x8e32ac56,
        0x2402f7f1,
        0x63fa4c5f,
        0x76697f3f,
        0x72b2adb1,
        0xa099e879,
        0xaeeb70e0,
        0xec6d9161,
        0xd7c688d7,
        0x13752ab9,
        0x1a11dffe,
        0x74de3c1c,
        0x9ac4b14a,
        0xf76ef546]

x_comm = [0x8f723116,
          0x6d7e4487,
          0xfb8ce824,
          0x54aae4c5,
          0xf6d1b0e7,
          0x19b57e17,
          0xf50afee0,
          0xd1a961ce]

In [12]:
out = verifiable_plus(4, rand, x_comm)
print(out)

12


As we see, this passes the check from the assert statement, so our input commitment matches the claimed commitment hash. Now we run this as a zero-knowledge proof

In [13]:
zkp.compile(verifiable_plus, includes=[commit, _3_plus_int], global_vars=globals()) 
crs = zkp.generate_crs(verifiable_plus)
zkp.store_crs(verifiable_plus, crs) # optional if run locally
proof = zkp.prove(verifiable_plus, 4, rand, x_comm)
zkp.store_proof(verifiable_plus, proof) # optional if run locally
zkp.verify(verifiable_plus, 4, rand, x_comm, return_value=out)

True

Note that sha256 is expensive to convert to R1CS, so in practice we will use a cheaper commitment scheme

Now suppose that the verifier received a corrupted input commitment

In [14]:
x_comm[7] = 0xd1a961cf

Let us prepare a new verification statement and then check the zero-knowledge proof

In [15]:
zkp.verify(verifiable_plus, 4, rand, x_comm, return_value=out)

False

As we can see, this did not pass the check, so it is a robust way to ensure that computations are correct w.r.t to their input commitments.

In [16]:
zkp.cleanup()

Lastly, let us also compute zero-knowledge proofs for the pedersen-hash and poseidon-hash functions

In [17]:
from zkpytoolkit.stdlib.commitment.pedersen.bls12_381.commit import commit_int as pedersen

def main(input: Private[Array[int, 16]], rand: Private[Array[int, 16]]) -> Array[int, 8]:
    hash: Array[int, 8] = pedersen(input, rand)
    return hash

In [18]:
input = [1 for _ in range(16)]
output = main(input, rand)
print('input: {}'.format(input))
print('hash: {}'.format(output))
constraints = zkp.compile(main, includes=[pedersen], global_vars=globals())
crs = zkp.generate_crs(main)
zkp.store_crs(main, crs) # optional if run locally
proof = zkp.prove(main, input, rand)
zkp.store_proof(main, proof) # optional if run locally
out = zkp.verify(main, None, None, return_value=output)
print(out)
print('#constraints: {}'.format(constraints))

input: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
hash: [892963126, 1086644132, 2482101191, 2670164535, 2255697141, 1082883849, 687455401, 1746318010]
True
#constraints: 8358


In [19]:
zkp.cleanup()

Finally we run an example for the poseidon hash

In [20]:
from zkpytoolkit.stdlib.hashes.poseidon.hash import hash

# let N = 6 for now
def main(inputs: Private[Array[field, 6]]) -> field:
    out: field = hash(inputs)
    return out

In [21]:
input = [12345678900000000101010101010 for _ in range(6)]
output = main(input)
print('input: {}'.format(input))
print('hash: {}'.format(output))
constraints = zkp.compile(main, includes=[hash], global_vars=globals()) 
crs = zkp.generate_crs(main)
zkp.store_crs(main, crs) # optional if run locally
proof = zkp.prove(main, input)
zkp.store_proof(main, proof) # optional if run locally
out = zkp.verify(main, return_value=output)
print(out)
print('#constraints: {}'.format(constraints))

input: [12345678900000000101010101010, 12345678900000000101010101010, 12345678900000000101010101010, 12345678900000000101010101010, 12345678900000000101010101010, 12345678900000000101010101010]
hash: 26214379981526208288684581651932840580862893297160543565534219724003945169923
True
#constraints: 354


In [22]:
zkp.cleanup()